In [1]:
import autogen
from autogen import ConversableAgent

In [2]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-5-mini","api_key":OPENAI_API_KEY}

In [2]:
from autogen import UserProxyAgent

In [4]:
import pprint

In [5]:
key_clause_extractor = ConversableAgent( name = "key_clause_extractor_Agent",
                                         system_message=""" You are an key clause extractor agent, your role is to review the legal content submitted and 
                                         extract the key clauses from the content""",
                                         llm_config=llm_config,
                                         human_input_mode="NEVER")

Risk_Flagger= ConversableAgent( name= "Risk_flagger_Agent",
                                system_message=""" You are an Risk Flagger Agent, your role is to flag any risk or harmful contents from the legal
                                contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

Final_reviewer= ConversableAgent( name= "Final_reviewer_Agent",
                                system_message=""" Final_reviewer_Agent, your role is to verify the entire legal content, and suggest any improvements if required 
                                and highlight any risky contents""",
                                llm_config=llm_config,
                                human_input_mode="NEVER")

In [6]:
User_proxy= UserProxyAgent( name ="UserProxyAgent",
                            human_input_mode="ALWAYS",
                            code_execution_config=False)
                              

In [7]:
cordinator= ConversableAgent( name = "cordinator_Agent",
                                 system_message="""Your role is like a mediator who gets the legal content from user proxy and pass it on to individual agents
                                 which will process the content further""",
                                 llm_config=llm_config,
                                 human_input_mode="NEVER")

In [3]:
human_counsel = UserProxyAgent(
    name="Human_Legal_Counsel",
    human_input_mode="ALWAYS",
    code_execution_config=False,
    description="Senior Counsel who intervenes only when AI fails."
)

message = """The Receiving Party agrees to treat all non-public information received
    from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.
    The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.
    This obligation does not apply to information that is publicly known through no fault of the Receiving Party.
    Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials."""

In [8]:
# the messages argument in message function holds the conversation summary between the cordinator an user proxy 
# and it does not append the next conversations automatically unless its done manually
# conversation thread between each agent is seperate

In [9]:
def message(recipient, messages, sender, config):
    """
    Robustly retrieves the input document from the current chat history.
    Takes 4 arguments as required by AutoGen's internal caller.
    """
    if not messages:
        return "No input provided. Please provide the document to review."
    
    # We take the last message from the coordinator's history with the user
    # to ensure the sub-agents are working on the most recent input.
    last_msg = messages[-1].get("content", "")
    return last_msg

In [10]:
def check_and_transfer(recipient, messages, sender, config):
    """
    Failure detection sensor. 
    If the last specialist failed, it sends a signal to trigger the human.
    """
    if not messages:
        return "FAILURE_SIGNAL: No messages received."
    
    last_reply = messages[-1].get("content", "")
    
    # Logic: detect if output is too short or contains error keywords
    if len(last_reply) < 20 or "error" in last_reply.lower() or "not found" in last_reply.lower():
        return f"FAILURE_SIGNAL: {sender.name} encountered an issue."
    
    return last_reply

def trigger_human_intervention(recipient, messages, sender, config):
    """
    The Escalation Brain.
    Listens to the Coordinator and interrupts if it sees 'FAILURE_SIGNAL'.
    """
    last_message = messages[-1].get("content", "")
    
    if "FAILURE_SIGNAL" in last_message:
        print(f"\n--- [SYSTEM ALERT: ESCALATING TO HUMAN DUE TO: {last_message}] ---\n")
        
        # Coordinator initiates an emergency chat with the Human
        recipient.initiate_chat(
            human_counsel,
            message=f"The automated workflow failed: {last_message}. Please provide the analysis manually to continue."
        )
        
        # Retrieve the human's input to override the failure
        human_input = recipient.last_message(human_counsel)["content"]
        return True, human_input # Returns (True, content) to override the AI reply
    
    return False, None

# 4. Registration

# Register the escalation logic to the Coordinator
cordinator.register_reply(
    [autogen.Agent, None],
    reply_func=trigger_human_intervention,
    position=0
)

# The Chain
chat_queue = [
    {
        "sender": cordinator,
        "recipient": key_clause_extractor,
        "message": message, # Uses the document loader function from previous step
        "summary_method": "last_msg",
        "max_turns": 1,
    },
    {
        "sender": cordinator,
        "recipient": Risk_Flagger,
        "message": check_and_transfer, # Inspects Extractor output
        "summary_method": "last_msg",
        "max_turns": 1,
    },
    {
        "sender": cordinator,
        "recipient": Final_reviewer,
        "message": check_and_transfer, # Inspects Risk output
        "carryover": [0, 1], # Feeds summaries to the human-like final review
        "max_turns": 1,
    }
]

In [11]:
cordinator.register_nested_chats(chat_queue, trigger= User_proxy)

In [12]:
#from autogen import initiate_chat

chats= cordinator.initiate_chat(User_proxy, message=" Please provide the legal content", max_turns=2)



cordinator_Agent (to UserProxyAgent):

 Please provide the legal content

--------------------------------------------------------------------------------


Replying as UserProxyAgent. Provide feedback to cordinator_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.


UserProxyAgent (to cordinator_Agent):

The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
cordinator_Agent (to key_clause_extractor_Agent):

The Receiving Party agrees to treat all non-

Replying as UserProxyAgent. Provide feedback to cordinator_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  0k


UserProxyAgent (to cordinator_Agent):

0k

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (cc55b32e-d789-4478-b1c5-9700a9136390): Maximum turns (2) reached


In [13]:
cordinator.chat_messages

defaultdict(list,
            {<autogen.agentchat.user_proxy_agent.UserProxyAgent at 0x1d98ea7d050>: [{'content': ' Please provide the legal content',
               'role': 'assistant',
               'name': 'cordinator_Agent'},
              {'content': 'The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
               'role': 'user',
               'name': 'UserProxyAgent'},
              {'content': "Good, your review covers most of the importa

In [14]:
cordinator.chat_messages_for_summary(User_proxy)

[{'content': ' Please provide the legal content',
  'role': 'assistant',
  'name': 'cordinator_Agent'},
 {'content': 'The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
  'role': 'user',
  'name': 'UserProxyAgent'},
 {'content': "Good, your review covers most of the important issues. I'll verify, tighten some points, add any items I think are missing, and give short model-language snippets you can drop into the clause. I’ll flag the highest legal/pr

In [15]:
cordinator.chat_messages_for_summary(User_proxy)[-1]['content']

'0k'

In [16]:
dir(chat_queue)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [17]:
chats.chat_history

[{'content': ' Please provide the legal content',
  'role': 'assistant',
  'name': 'cordinator_Agent'},
 {'content': 'The Receiving Party agrees to treat all non-public information received     from the Disclosing Party as strictly confidential. This includes technical data, business plans, and client lists.     The Receiving Party shall not disclose, publish, or use this information for any purpose other than evaluating the proposed business relationship without prior written consent.     This obligation does not apply to information that is publicly known through no fault of the Receiving Party.     Upon termination of discussions, the Receiving Party shall return or destroy all confidential materials.',
  'role': 'user',
  'name': 'UserProxyAgent'},
 {'content': "Good, your review covers most of the important issues. I'll verify, tighten some points, add any items I think are missing, and give short model-language snippets you can drop into the clause. I’ll flag the highest legal/pr